In [1]:
# LOAD PACKAGES
import numpy as np

In [60]:
# UPLOAD DATA
# (each user should put datafiles in this directory on their computer)
datapath = "blast_tab_1hit.out"
file = open(datapath, 'r')
entries = []
num_same = 0
for ln in file:
    toks = ln.split('\t')
    if toks[2] == toks[3]:
        num_same += 1
    entries.append(toks)
file.close()
num_entries = len(entries)

In [61]:
# ANALYZE DATA
print(str(num_entries) + ' sequences were uploaded')
print(str(num_same) + ' sequences had the same input and output')
print(str(num_entries - num_same) + ' had errors')
print('\nExample:\n')
print(entries[0])

490670 sequences were uploaded
431807 sequences had the same input and output
58863 had errors

Example:

['ERR777695.8', 'Desulfovibrio_piger_ATCC_29098_3', 'ACGGAGGGTGCGAGCGTTAATCGGAATCACTGGGCGTAAAGCGCACGTAGGCTGCTTGGTAAGTCAGGGGTGAAAGCCCGCGGCTCAACCGCGGAATTGCCTTTGATACTGC-CGAGCTAGAGTCCGGGAGAGGGTAGTGGAATTCCAGGTGTAGGAGTGAAATCCGTAGAGATCTGGAGGAACATCAGTGGCGAAGGCGACTACCTGGACCGGTACTGACGCTGAGGTGCGAAAGCGTGGGGAGCAAACAG', 'ACGGAGGGTGCAAGCGTTAATCGGAATCACTGGGCGTAAAGCGCACGTAGGCTGTTATGTAAGTCAGGGGTGAAAGCCCACGGCTCAACCGTGGAACTGCCCTTGATACTGCACGA-CTCGAATCCGGGAGAGGGTGGCGGAATTCCAGGTGTAGGAGTGAAATCCGTAGATATCTGGAGGAACATCAGTGGCGAAGGCGGCCACCTGGACCGGTATTGACGCTGAGGTGCGAAAGCGTGGGGAGCAAACAG', '5.64e-103\n']
